In [15]:
### CRIANDO BERTOPICS PARA DESCRIÇAO DAS ONGS: 

import pandas as pd
from bertopic import BERTopic
from elasticsearch import Elasticsearch, helpers, exceptions
from elasticsearch.helpers import scan as escan
import collections
from sentence_transformers import SentenceTransformer
from bertopic.representation import KeyBERTInspired
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer



Carregando os pacotes necessários no python, utilizaremos o Bertopics para agrupar as descrições das ONGs.

BERTopic é uma técnica de modelagem de tópicos que usa embeddings do BERT para criar agrupamentos densos e interpretáveis que podem ser descritos pelas palavras mais importantes. 

Na primeira etapa o BERTopic busca representar os documentos a partir de vetores de números que carregam o significado de palavras e textos. Estes embeddings podem ser pré-treinados em grandes conjuntos de textos, garantindo maiores chances de capturar questões como silimaridade, sinônimos, antônimos e etc...

Com os documentos representados numericamente, o BERTopic começaa a agrupá-los, utilizando um medida chamada de c-TF-IDF. Esta medida combina a frequência de uma palavra em um documento com a sua importância global. As palavras que aparecem com frequência em um documento e são importantes em geral são mais propensas a serem representativas desse documento.

O c-TF-IDF é, por fim, utilizado para calcular uma pontuação para cada documento. Documentos com pontuações semelhantes são agrupados em um mesmo tópico.

A última etapa do BERTopic é gerar etiquetas descritivas para cada tópico. Para isso, BERTopic identifica as palavras mais importantes de cada tópico que são aquelas que têm as pontuações mais altas no c-TF-IDF.


### Carregando o banco de dados com a descrição das ONGS: 

Utilizamos aqui dados de [ONGS](https://www.globalgiving.org/) e sua breve descrição em texto.


In [3]:
#### Carregando o banco de dados

df = pd.read_csv("data/banco_ongs_join.csv")

text = df["descricao"]


### Definições

- A nossa primeira definição é o uso de um modelo de embeddings, no [MTEB leaderboard](https://huggingface.co/spaces/mteb/leaderboard) temos alguns modelos e podemos escolher, inclusive modelos multilinguais; 

- Redução de dimensionalidade: Optamos por usar o [UMAP](https://arxiv.org/abs/1802.03426) que é o padrão do BERTopic;

- Clustering: Utilizamos o método de HDBSCAN que é o padrão para o BERTopic;



In [16]:
### BERTOPICS: 

#### Embeddings
sentence_model = SentenceTransformer("all-MiniLM-L6-v2")

#### Redução de dimensionalidade

umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine')

### CLUSTERING: 

hdbscan_model = HDBSCAN(min_cluster_size=15, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

### REMOVENDO STOPWORDS: 

vectorizer_model = CountVectorizer(ngram_range=(1, 3), stop_words="english")

### Rodando modelo

topic_model = BERTopic(embedding_model=sentence_model, umap_model=umap_model, hdbscan_model=hdbscan_model, 
                       vectorizer_model=vectorizer_model, 
                       nr_topics="auto")


topics, probs = topic_model.fit_transform(text)



### Observando os resultados: 

Infelizmente, ainda temos um número grande de descrições no Tópico 0, o que indica a necessidade de melhorar os parametros da modelagem, escolhendo outra forma de embedding, determinando palavras-chaves a priori e coisas assim.

Apesar disto, temos alguns grupos bem definidos: 

- Um deles referentes a questões marinhas, marcado por palavras como corais, recifes e peixes;
- Um deles ligados a segurança energética, trazendo questões como energia solar;
- Um deles ligados a questões de limitações visuais;
- Um deles ligadas a questão das tartarugas;
- Um deles ligado a educação e mudança de vida através do esporte; e 
- Outro ligado a crianças inclusas no espectro autista.

In [18]:
topic_model.get_topic_info()


,Topic,Count,Name,Representation,Representative_Docs
0,-1,1071,-1_children_education_school_help,"[children, education, school, help, project, g...","[The Fundacion Ninos de los Andes (FNA), is wo..."
1,0,4140,0_children_project_education_school,"[children, project, education, school, support...","[This project provides food, medicines, and a ..."
2,1,190,1_li_public health_bauchi_public,"[li, public health, bauchi, public, working, h...",[As members of Leadership Initiatives' (LI) In...
3,2,42,2_ocean_marine_coral_reef,"[ocean, marine, coral, reef, local, reefs, fis...",[Reef Check Malaysia works to conserve Malaysi...
4,3,35,3_solar_light_bangladesh_kerosene,"[solar, light, bangladesh, kerosene, lamps, 00...","[This project will supply 20,000 solar lamps i..."
5,4,28,4_blindness_eye_vision_sight,"[blindness, eye, vision, sight, cataract, blin...",[This project will give the gift of SIGHT to10...
6,5,23,5_sea_turtles_turtle_sea turtle,"[sea, turtles, turtle, sea turtle, eggs, curac...",[Sea Turtle Conservation Curacao does research...
7,6,22,6_arche_die arche_die_children,"[arche, die arche, die, children, inner cities...","[""Making children strong for life!"" This sloga..."
8,7,19,7_september_earthquake_high atlas_atlas,"[september, earthquake, high atlas, atlas, mag...","[On 8th September 2023, Morocco was hit hard b..."
9,8,19,8_sport_life skills_sports_life,"[sport, life skills, sports, life, skills, spo...",[Football can change individuals AND communiti...


### Visualizando: 

Podemos ver na visualização que há espaço para melhorar o Tópico 0, quebrando em grupos menores e possibilitando melhor entendimento dos grupos de documentos.

In [19]:
topic_model.visualize_topics()
